In [58]:
import sys
import os

%load_ext autoreload
%autoreload 2

import zarr
import numpy as np
import torch
import copy
import numcodecs
import shortuuid

from pathlib import Path

import gym_pusht  # noqa: F401
import gymnasium as gym
import imageio
import numpy
import torch
from huggingface_hub import snapshot_download

from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy

from lerobot.common.datasets.push_dataset_to_hub.utils import (
    concatenate_episodes,
    get_default_encoding,
    save_images_concurrently,
)
from lerobot.common.datasets.utils import (
    calculate_episode_data_index,
    hf_transform_to_torch,
)
from lerobot.common.datasets.lerobot_dataset import CODEBASE_VERSION
from IPython.display import Video
from lerobot.common.datasets.rollout_datasets.episode_stores import EpisodeVideoStore

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
output_directory = Path("/home/sm/Datasets/lerobot/pusht")
output_directory.mkdir(parents=True, exist_ok=True)

videos_dir = output_directory / 'videos'
videos_dir.mkdir(parents=True, exist_ok=True)

In [60]:
episode_video_store = EpisodeVideoStore.create_from_path(output_directory, mode='r')

Connected to the exising zarr


In [61]:
dataset = episode_video_store.convert_to_lerobot_dataset(repo_id='place/holder')

print(f"Number of episodes: {dataset.num_episodes}")
print(f"\naverage number of frames per episode: {dataset.num_samples / dataset.num_episodes:.3f}")
print(f"frames per second used during data collection: {dataset.fps=}")
print(f"keys to access images from cameras: {dataset.camera_keys=}\n")

Generating train split: 475145 examples [04:08, 1910.67 examples/s]


Number of episodes: 2010

average number of frames per episode: 236.391
frames per second used during data collection: dataset.fps=10
keys to access images from cameras: dataset.camera_keys=['observation.image']



In [55]:
episode_index = 3


from_idx = dataset.episode_data_index["from"][episode_index].item()
to_idx = dataset.episode_data_index["to"][episode_index].item()

print(f"episode {episode_index} start from index {from_idx} to index {to_idx}")

frames = [dataset[idx]["observation.image"] for idx in range(from_idx, to_idx)]

# Video frames are now float32 in range [0,1] channel first (c,h,w) to follow pytorch convention. To visualize
# them, we convert to uint8 in range [0,255]
frames = [(frame * 255).type(torch.uint8) for frame in frames]
# and to channel last (h,w,c).
frames = [frame.permute((1, 2, 0)).numpy() for frame in frames]

Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
video_path = f"outputs/examples/1_load_lerobot_dataset/episode_{episode_index}.mp4"
imageio.mimsave(video_path, frames, fps=dataset.fps)

# 비디오 표시
Video(video_path, embed=True, width=640, height=360)

episode 3 start from index 893 to index 1188


In [49]:
delta_timestamps = {
    # loads 4 images: 1 second before current frame, 500 ms before, 200 ms before, and current frame
    "observation.image": [-1, -0.5, -0.20, 0],
    # loads 8 state vectors: 1.5 seconds before, 1 second before, ... 20 ms, 10 ms, and current frame
    "observation.state": [-1.5, -1, -0.5, -0.20, -0.10, -0.02, -0.01, 0],
    # loads 64 action vectors: current frame, 1 frame in the future, 2 frames, ... 63 frames in the future
    "action": [t / dataset.fps for t in range(64)],
}

In [50]:
delta_dataset = episode_video_store.convert_to_lerobot_dataset(repo_id='place/holder', delta_timestamps=delta_timestamps)

Generating train split: 1488 examples [00:00, 2114.12 examples/s]


In [51]:
dataloader = torch.utils.data.DataLoader(
    delta_dataset,
    num_workers=0,
    batch_size=32,
    shuffle=True,
)
for batch in dataloader:
    print(f"{batch['observation.image'].shape=}")  # (32,4,c,h,w)
    print(f"{batch['observation.state'].shape=}")  # (32,8,c)
    print(f"{batch['action'].shape=}")  # (32,64,c)
    break

batch['observation.image'].shape=torch.Size([32, 4, 3, 688, 688])
batch['observation.state'].shape=torch.Size([32, 8, 2])
batch['action'].shape=torch.Size([32, 64, 2])
